# Q1

In [1]:
from voc_dataset import VOCDataset
import utils
import os
import xml.etree.ElementTree as ET
import torch

/home/ubuntu/anaconda3/envs/cmu16824hw1/lib/python3.7/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
data_dir = '/mnt/data/VOCdevkit/VOC2007/'
ann_dir = os.path.join(data_dir, 'Annotations')

In [3]:
split='trainval'
split_file = os.path.join(data_dir, 'ImageSets/Main', split + '.txt')
with open(split_file) as fp:
    index_list = [line.strip() for line in fp]

In [4]:
index = index_list[0]
fpath = os.path.join(ann_dir, index + '.xml')
tree = ET.parse(fpath)

In [5]:
root = tree.getroot() # Tree root
class_lst = []
for child in root:
    if child.tag == 'object':
        child_lst = child.getchildren()
        for item in child_lst:
            if item.tag == 'name':
                name = item.text
            elif item.tag == 'difficult':
                difficulty = item.text
        class_lst.append((name, difficulty))

/home/ubuntu/anaconda3/envs/cmu16824hw1/lib/python3.7/site-packages/ipykernel_launcher.py:5: DeprecationWarning: This method will be removed in future versions.  Use 'list(elem)' or iteration over elem instead.
  """


In [6]:
index

'000005'

In [7]:
class_lst

[('chair', '0'),
 ('chair', '0'),
 ('chair', '1'),
 ('chair', '0'),
 ('chair', '1')]

In [8]:
CLASS_NAMES = ['aeroplane', 'bicycle', 'bird', 'boat', 'bottle', 'bus', 'car',
                   'cat', 'chair', 'cow', 'diningtable', 'dog', 'horse', 'motorbike',
                   'person', 'pottedplant', 'sheep', 'sofa', 'train', 'tvmonitor']

In [9]:
class_vec = torch.zeros(20)
weight_vec = torch.ones(20)
for item in class_lst:
    idx = CLASS_NAMES.index(item[0])
    if class_vec[idx] == 1:
        weight_vec[idx] = 0 if item[1] == '1' else 1
        continue
    class_vec[idx] = 1
    weight_vec[idx] = 0 if item[1] == '1' else 1

print(class_vec)
print(weight_vec)

tensor([0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0.])
tensor([1., 1., 1., 1., 1., 1., 1., 1., 0., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1.])


In [12]:
def _calculate_conv_out_size(inp_size, padding, dilation, kernel_size, stride):
        return (inp_size + 2*padding - dilation * (kernel_size-1) - 1)//stride + 1
    
def _calculate_avg_pool_out_size(inp_size, padding, kernel_size, stride):
    return (inp_size + 2*padding - kernel_size)//stride + 1

In [18]:
out_size1 = _calculate_conv_out_size(64, 2, 1, 5, 1)
out_size2 = _calculate_avg_pool_out_size(out_size1, 0, 2, 2)
out_size3 = _calculate_conv_out_size(out_size2, 2, 1, 5, 1)
out_size4 = _calculate_avg_pool_out_size(out_size3, 0, 2, 2)
flat_dim = 64 * out_size4 * out_size4
print(flat_dim)

16384


In [40]:
import torch
import torch.nn.functional as F

torch.manual_seed(42)
# logits: [N, C] (raw predictions from the model)
# targets: [N, C] (one-hot encoded true labels)
N = 3
C = 20
logits = torch.randn(N, C)  # example logits
targets = torch.randint(0, 2, (N, C)).float()  # example one-hot encoded targets
weights = torch.randint(0, 2, (N, C)).float()  # example one-hot encoded targets

# Calculate the softmax probabilities
# softmax_probs = F.softmax(logits, dim=1)
softmax_denom = torch.sum(torch.exp(logits-torch.max(logits)), dim=1, keepdims=True)
softmax_probs = torch.exp(logits-torch.max(logits)) / softmax_denom

# Compute cross-entropy loss
loss = -torch.sum(weights * torch.log(softmax_probs) * targets, dim=1).mean()  # per sample
print(loss)

tensor(15.8957)


In [2]:
CLASS_NAMES = ['aeroplane', 'bicycle', 'bird', 'boat', 'bottle', 'bus', 'car',
                   'cat', 'chair', 'cow', 'diningtable', 'dog', 'horse', 'motorbike',
                   'person', 'pottedplant', 'sheep', 'sofa', 'train', 'tvmonitor']
INV_CLASS = {}
for i in range(len(CLASS_NAMES)):
    INV_CLASS[CLASS_NAMES[i]] = i

In [3]:
INV_CLASS

{'aeroplane': 0,
 'bicycle': 1,
 'bird': 2,
 'boat': 3,
 'bottle': 4,
 'bus': 5,
 'car': 6,
 'cat': 7,
 'chair': 8,
 'cow': 9,
 'diningtable': 10,
 'dog': 11,
 'horse': 12,
 'motorbike': 13,
 'person': 14,
 'pottedplant': 15,
 'sheep': 16,
 'sofa': 17,
 'train': 18,
 'tvmonitor': 19}

# Q2

In [1]:
import torch
import trainer
from utils import ARGS
from simple_cnn import SimpleCNN
from voc_dataset import VOCDataset
import numpy as np
import torchvision
import torch.nn as nn
import random

/home/ubuntu/anaconda3/envs/cmu16824hw1/lib/python3.7/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [24]:
model = torchvision.models.resnet18(weights='IMAGENET1K_V1')

In [25]:
model.fc = nn.Linear(in_features=512, out_features=20)
print(model.fc)

Linear(in_features=512, out_features=20, bias=True)


In [26]:
for param in model.parameters():
    param.requires_grad = False

In [23]:
for param in model.fc.parameters():
    print(param)

Parameter containing:
tensor([[ 0.0189,  0.0221, -0.0231,  ...,  0.0372, -0.0079, -0.0428],
        [-0.0165, -0.0285, -0.0409,  ..., -0.0014, -0.0223, -0.0177],
        [-0.0172, -0.0369, -0.0080,  ..., -0.0182, -0.0387,  0.0408],
        ...,
        [-0.0314, -0.0052,  0.0127,  ..., -0.0377,  0.0421, -0.0220],
        [-0.0191, -0.0435,  0.0027,  ...,  0.0041, -0.0259, -0.0382],
        [ 0.0354, -0.0235, -0.0297,  ..., -0.0293,  0.0349, -0.0180]],
       requires_grad=True)
Parameter containing:
tensor([-0.0159, -0.0069, -0.0341, -0.0436, -0.0022, -0.0198, -0.0345,  0.0200,
         0.0137, -0.0033,  0.0294,  0.0169, -0.0413,  0.0085, -0.0097, -0.0170,
        -0.0102, -0.0223, -0.0293, -0.0434], requires_grad=True)
